#### copied from Doug's work - to apply to my NYC data

In [1]:
import numpy as np
import pandas as pd
#if csvs are in same directory as notebook, set filepath = './'
filepath = input('Enter file path where csv files are located on your machine\n')

# my filepath is ~/Desktop/NYCDSA/Capstone/data/
multi = pd.read_csv(filepath + '/Zip_Zri_MultiFamilyResidenceRental.csv',dtype = {'RegionName':str})

Enter file path where csv files are located on your machine
~/Desktop/NYCDSA/Capstone/data


In [2]:

def zero_zips(df):
    '''Adds zeros to beginning of zip codes that were chopped off when of int dtype
    
    Args:
    
    df: name of the DataFrame'''
    return df['RegionName'].apply(lambda Z: '0'*(5-len(Z))+Z)

multi['RegionName'] = zero_zips(multi)
# homes['RegionName'] = zero_zips(homes)
# single['RegionName'] = zero_zips(single)

print(multi.shape)
# print(single.shape)
# print(homes.shape)

(1861, 120)


In [3]:
multi = pd.melt(multi, id_vars =multi.columns[:7],
                value_vars = multi.columns[7:],
               var_name = 'Date',value_name = 'Rent')
multi['Date'] = pd.to_datetime(multi['Date'])
multi.rename(columns = {'RegionName':'Zipcode'}, inplace = True)



multi.sample(10)

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Date,Rent
168332,73051,34232,Sarasota,FL,North Port-Sarasota-Bradenton,Sarasota County,843,2018-03-01,1435.0
1185,87129,66213,Overland Park,KS,Kansas City,Johnson County,1186,2010-09-01,NaN
60219,95997,90016,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,668,2013-05-01,1674.0
30302,69452,27612,Raleigh,NC,Raleigh,Wake County,527,2012-01-01,NaN
153164,81295,53215,Milwaukee,WI,Milwaukee-Waukesha-West Allis,Milwaukee County,563,2017-07-01,741.0
136392,96886,92507,Riverside,CA,Riverside-San Bernardino-Ontario,Riverside County,540,2016-10-01,1322.0
147775,72497,33196,The Hammocks,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,757,2017-04-01,1556.0
127436,97850,94804,Richmond,CA,San Francisco-Oakland-Hayward,Contra Costa County,889,2016-05-01,2283.0
142024,61801,10461,New York,NY,New York-Newark-Jersey City,Bronx County,589,2017-01-01,1702.0
21267,79779,49503,Grand Rapids,MI,Grand Rapids-Wyoming,Kent County,797,2011-08-01,NaN


### Select NYC data

In [4]:
nyc = multi.loc[(multi['City'] == 'New York') & (multi['State']=='NY')].set_index('Date')

nyc.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Rent
Date,,,,,,,,
2010-09-01,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2930.0
2010-09-01,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2797.0
2010-09-01,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,7,NaN
2010-09-01,62037,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,1717.0
2010-09-01,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,12,1347.0


In [12]:
nyc.CountyName.value_counts()

Queens County      5537
Kings County       4181
New York County    3842
Bronx County       2034
Richmond County    1356
Name: CountyName, dtype: int64

## Impute for each zip code

In [13]:
fill_rents = nyc[['Zipcode','Rent']].copy()
nyc.drop('Rent', axis = 1, inplace = True)
fill_rents = fill_rents.reset_index().pivot(index = 'Date',columns = 'Zipcode')['Rent'].reset_index()
# for code in fill_rents.columns[1:]:
#     fill_rents[code].interpolate(inplace = True)
    
fill_rents.fillna(method = 'bfill',inplace = True)

fill_rents.isnull().sum()

fill_rents

Zipcode,Date,10001,10002,10003,10005,10009,10010,10011,10012,10013,...,11428,11429,11432,11433,11434,11435,11436,11691,11692,11694
0,2010-09-01,2869.0,3083.0,2884.0,2831.0,3062.0,2875.0,2783.0,2742.0,3791.0,...,1685.0,1414.0,1425.0,1545.0,1530.0,1443.0,1442.0,1507.0,1536.0,1509.0
1,2010-10-01,2874.0,3083.0,2916.0,2900.0,3052.0,2926.0,2781.0,2794.0,4046.0,...,1685.0,1422.0,1443.0,1549.0,1530.0,1458.0,1452.0,1540.0,1536.0,1520.0
2,2010-11-01,2867.0,3083.0,2936.0,2966.0,3014.0,2943.0,2764.0,2830.0,4046.0,...,1685.0,1445.0,1464.0,1548.0,1533.0,1472.0,1463.0,1550.0,1537.0,1529.0
3,2010-12-01,2842.0,3083.0,2926.0,3015.0,2980.0,2928.0,2717.0,2824.0,4141.0,...,1685.0,1456.0,1480.0,1547.0,1539.0,1483.0,1471.0,1557.0,1540.0,1524.0
4,2011-01-01,2792.0,3083.0,2891.0,2982.0,2918.0,2887.0,2666.0,2768.0,4161.0,...,1685.0,1457.0,1481.0,1544.0,1543.0,1471.0,1489.0,1558.0,1532.0,1523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2019-09-01,3696.0,3481.0,3672.0,3738.0,3787.0,3758.0,3805.0,3526.0,4501.0,...,2040.0,2070.0,2005.0,2173.0,2175.0,2000.0,2086.0,2185.0,2145.0,2159.0
109,2019-10-01,3701.0,3437.0,3641.0,3729.0,3753.0,3782.0,3747.0,3492.0,4488.0,...,2038.0,2090.0,2017.0,2199.0,2203.0,1991.0,2084.0,2209.0,2183.0,2177.0
110,2019-11-01,3628.0,3485.0,3593.0,3674.0,3798.0,3810.0,NaN,3480.0,4599.0,...,2052.0,2113.0,2026.0,2209.0,2224.0,1975.0,2119.0,2250.0,2200.0,2176.0
111,2019-12-01,3717.0,3538.0,3686.0,3765.0,3851.0,3817.0,NaN,NaN,NaN,...,2053.0,2111.0,1990.0,2212.0,2216.0,1963.0,2115.0,2227.0,2203.0,2133.0


In [14]:
fill_rents = pd.melt(fill_rents, id_vars='Date', value_vars = fill_rents.columns[1:],var_name='Zipcode',value_name = 'Rent')

fill_rents.head()

,Date,Zipcode,Rent
0,2010-09-01,10001,2869.0
1,2010-10-01,10001,2874.0
2,2010-11-01,10001,2867.0
3,2010-12-01,10001,2842.0
4,2011-01-01,10001,2792.0


In [15]:
nyc = pd.merge(nyc, fill_rents, on = ['Date','Zipcode'])

In [16]:
nyc.head()

,Date,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Rent
0,2010-09-01,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2930.0
1,2010-09-01,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2797.0
2,2010-09-01,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,7,3083.0
3,2010-09-01,62037,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,1717.0
4,2010-09-01,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,12,1347.0


## load census data

In [26]:
acs = pd.read_csv('~/Desktop/NYCDSA/Capstone/data/census_feats.csv', dtype = {'zip_code':str})
# total units for housing in zip code / total population
acs['per_person_housing'] = acs['housing_units'] / acs['total_pop'] 
# total commute time for people in zip / total population
acs['mean_commute_time'] = acs['aggregate_travel_time_to_work'] / acs['total_pop']
# number of people in the zip code with a bachelors or higher / total population
acs['bachelors_ormore'] = acs['bachelors_degree_or_higher_25_64'] / acs['total_pop']
acs = acs.drop('housing_units',axis=1).drop('aggregate_travel_time_to_work',axis=1).\
drop('bachelors_degree_or_higher_25_64',axis=1)

In [37]:
nyc_census = nyc.merge(acs,how='left',left_on='Zipcode',right_on='zip_code')


In [38]:
nyc_census.drop(['Metro','RegionID','City','State','geo_id','zip_code'],axis=1,inplace=True)
nyc_census.sample(10)

,Date,Zipcode,CountyName,SizeRank,Rent,total_pop,gini_index,median_income,per_person_housing,mean_commute_time,bachelors_ormore
14867,2018-12-01,10003,New York County,50,3661.0,54682.0,0.5571,112131.0,0.564647,13.620021,0.483706
3941,2012-11-01,10457,Bronx County,163,1416.0,74554.0,0.5047,27420.0,0.346957,15.344717,0.066462
1437,2011-06-01,11237,Kings County,868,1609.0,49310.0,0.4731,51413.0,0.364774,19.771040,0.195538
389,2010-11-01,11103,Queens County,909,1604.0,36140.0,0.4263,71437.0,0.498506,23.286801,0.390205
16687,2019-12-01,10468,Bronx County,140,NaN,78647.0,0.4779,36046.0,0.349854,19.327565,0.080092
12762,2017-10-01,10128,New York County,31,3093.0,59256.0,0.5586,114010.0,0.587114,18.897833,0.529955
13667,2018-04-01,10003,New York County,50,3608.0,54682.0,0.5571,112131.0,0.564647,13.620021,0.483706
16128,2019-08-01,11222,Kings County,597,2890.0,36492.0,0.4477,85111.0,0.511235,23.117533,0.468267
9905,2016-03-01,10016,New York County,15,3530.0,51057.0,0.5099,117004.0,0.678418,15.994085,0.565094
12972,2017-11-01,10010,New York County,522,3714.0,33730.0,0.5366,117923.0,0.549511,15.892381,0.514438


In [39]:
nyc_df = nyc_census.sort_values(by = 'Date').set_index('Date')
nyc_df = nyc_df['2013-01-01':'2018-07-01']

In [64]:
nyc_df[nyc_df.isnull().sum(axis = 1)>0].Zipcode.value_counts()

11249    67
11109    67
10069    67
Name: Zipcode, dtype: int64

In [115]:
nyc_impute = nyc_df.fillna({'mean_commute_time':np.mean(nyc_df['mean_commute_time']),
               'total_pop':np.mean(nyc_df['total_pop']),
               'gini_index':np.mean(nyc_df['gini_index']),
               'median_income':np.mean(nyc_df['median_income']),
               'per_person_housing':np.mean(nyc_df.per_person_housing),
               'bachelors_ormore':np.mean(nyc_df.bachelors_ormore)
              })

In [116]:
nyc_impute.sample(10)

,Zipcode,CountyName,SizeRank,Rent,total_pop,gini_index,median_income,per_person_housing,mean_commute_time,bachelors_ormore
Date,,,,,,,,,,
2015-10-01,10465,Bronx County,1171,1619.0,44205.000000,0.445300,74796.000000,0.380998,17.620857,0.163398
2017-03-01,11432,Queens County,634,1931.0,62053.000000,0.471300,55906.000000,0.322789,20.208531,0.196074
2015-02-01,11249,Kings County,804,2911.0,51775.771812,0.483646,71804.637584,0.428451,18.689923,0.255043
2016-06-01,11233,Kings County,155,2252.0,76819.000000,0.537800,40449.000000,0.417709,18.203179,0.148388
2018-04-01,10314,Richmond County,111,1860.0,90761.000000,0.425400,86742.000000,0.366545,18.575985,0.197166
2016-03-01,10019,New York County,48,3542.0,43703.000000,0.566600,98833.000000,0.756859,16.464774,0.540215
2014-11-01,10128,New York County,31,2986.0,59256.000000,0.558600,114010.000000,0.587114,18.897833,0.529955
2015-11-01,11210,Kings County,458,1923.0,67040.000000,0.465300,65900.000000,0.359905,19.869406,0.190543
2018-04-01,10017,New York County,1307,3667.0,15801.000000,0.500000,127506.000000,0.800582,17.835580,0.576546


In [117]:
nyc_impute.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10050 entries, 2013-01-01 to 2018-07-01
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Zipcode             10050 non-null  object 
 1   CountyName          10050 non-null  object 
 2   SizeRank            10050 non-null  int64  
 3   Rent                10050 non-null  float64
 4   total_pop           10050 non-null  float64
 5   gini_index          10050 non-null  float64
 6   median_income       10050 non-null  float64
 7   per_person_housing  10050 non-null  float64
 8   mean_commute_time   10050 non-null  float64
 9   bachelors_ormore    10050 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 863.7+ KB


In [99]:
len(nyc_impute['Zipcode'].value_counts())

150

In [100]:
nyc_impute.isnull().sum(axis = 0)

Zipcode               0
SizeRank              0
Rent                  0
total_pop             0
gini_index            0
median_income         0
per_person_housing    0
mean_commute_time     0
bachelors_ormore      0
Bronx                 0
Kings                 0
New York              0
Richmond              0
dtype: int64

In [121]:
nyc_impute

,Zipcode,CountyName,SizeRank,Rent,total_pop,gini_index,median_income,per_person_housing,mean_commute_time,bachelors_ormore
Date,,,,,,,,,,
2013-01-01,10465,Bronx County,1171,1498.0,44205.0,0.4453,74796.0,0.380998,17.620857,0.163398
2013-01-01,10075,New York County,1008,2741.0,21556.0,0.5772,137146.0,0.584478,15.481305,0.431713
2013-01-01,10012,New York County,1043,3035.0,23411.0,0.5429,109037.0,0.600231,13.883004,0.546709
2013-01-01,11357,Queens County,1093,1695.0,39902.0,0.4216,83951.0,0.400306,18.134555,0.226355
2013-01-01,10305,Richmond County,1114,1322.0,42298.0,0.4267,76489.0,0.379876,18.457728,0.206156
...,...,...,...,...,...,...,...,...,...,...
2018-07-01,11101,Queens County,254,2413.0,30043.0,0.5389,65392.0,0.559664,18.195253,0.337916
2018-07-01,11204,Kings County,266,1992.0,76978.0,0.4850,49785.0,0.334277,16.379160,0.147782
2018-07-01,11213,Kings County,268,2282.0,66295.0,0.5060,41710.0,0.414315,17.328079,0.165261


# Predict using random forest

In [118]:
nyc_forest = nyc_impute.copy()
nyc_forest['Zipcode'] = nyc_forest['Zipcode'].astype(int)

from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor()

In [122]:
from sklearn.model_selection import train_test_split
y = np.log(nyc_forest['Rent'])
X = nyc_forest.drop(['Rent','CountyName'], axis = 1)

Xtrain,Xtest,ytrain,ytest = train_test_split(X,y)

In [123]:
rfc.fit(Xtrain,ytrain)
rfc.score(Xtest,ytest)

0.911724177192549

In [124]:
rfc.score(Xtrain,ytrain)

0.913682138960813

In [114]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [106]:
from sklearn.model_selection import cross_validate, KFold

cv = KFold(n_splits = 5, shuffle = True)

In [107]:
cv_res = cross_validate(RandomForestRegressor(),X,y, cv = cv)

In [112]:
cv_res2

{'fit_time': array([0.52205801, 0.54436493, 0.50615096, 0.51815772, 0.54785895]),
 'score_time': array([0.01896715, 0.01862025, 0.01712418, 0.01842022, 0.02053618]),
 'test_score': array([0.91081639, 0.90939862, 0.90540865, 0.90849956, 0.9137624 ])}

In [108]:
cv_res2 = cross_validate(RandomForestRegressor(),Xtrain,ytrain,cv = cv)

In [125]:
for col, imp in zip(X.columns,rfc.feature_importances_):
    print(f'{col}: {imp}')

Zipcode: 0.8673646013062734
SizeRank: 0.005184844217779788
total_pop: 0.004739200806532296
gini_index: 0.017484782209136598
median_income: 0.004195846088804284
per_person_housing: 0.025967895311745075
mean_commute_time: 0.008691690112099613
bachelors_ormore: 0.06637113994762905


# Try random forest using population from BigQuery

In [257]:
zipcodes = pd.read_csv('~/Downloads/zipcode_2.csv')

In [258]:
zipcodes = zipcodes[['geo_id','total_pop']]

aus_zip = zipcodes.loc[zipcodes['geo_id'].isin(aus_forest['Zipcode'])]

In [259]:
aus_zip

,geo_id,total_pop
3005,78751,15805
4320,78727,29509
5735,78701,9427
9390,78705,33948
9416,78752,21324
9617,78729,29315
10243,78744,48969
11172,78757,24823
15172,78731,27175
15181,78730,9186


In [270]:
aus_forest2 = pd.merge(aus_forest,aus_zip, left_on = 'Zipcode', right_on = 'geo_id')

In [261]:
aus_forest2.drop(['Population','geo_id'], axis = 1, inplace = True)

In [271]:
np.sum(aus_zip['total_pop'])

883523

In [273]:
aus_forest2['Population'].value_counts()

2002134.0    324
1883315.0    324
2062016.0    324
2115475.0    324
1942467.0    324
2165497.0    189
Name: Population, dtype: int64

In [262]:
y = np.log(aus_forest2['Rent'])
X = aus_forest2.drop('Rent', axis = 1)

Xtrain,Xtest,ytrain,ytest = train_test_split(X,y)

In [263]:
rfc = RandomForestRegressor()
rfc.fit(Xtrain,ytrain)
rfc.score(Xtest,ytest)

0.9922435214917491

In [264]:
rfc.score(Xtrain,ytrain)

0.9989321758284088

In [265]:
from sklearn.model_selection import cross_validate, KFold

cv = KFold(n_splits = 5, shuffle = True)

In [266]:
cv_res = cross_validate(RandomForestRegressor(),X,y, cv = cv)

In [267]:
cv_res2 = cross_validate(RandomForestRegressor(),Xtrain,ytrain,cv = cv)

In [268]:
for col, imp in zip(X.columns,rfc.feature_importances_):
    print(f'{col}: {imp}')

Zipcode: 0.5831951395527925
SizeRank: 0.10910395531283357
Low_income_volume: 0.014231952871825115
Moderate_income_volume: 0.024333255086773045
LaborForce: 0.10044627293835216
NumEmp: 0.044125247755545625
Income_per_capita: 0.004924093038672145
total_pop: 0.11964008344320598
